# Pretrain

1) Препроцессинг данных

In [1]:
import re
import json
import hashlib
from pathlib import Path

In [2]:
DATA_PATH = Path("./data/corpus")
OUT_PATH = Path("./data/pretrain_corpus.jsonl")

MIN_SENT_CHARS = 20
MAX_SENT_CHARS = 5000

CONTEXT_LEN = 1024
MAX_TOKENS_PER_CHUNK = CONTEXT_LEN - 2 

BOS = "<bos>"
EOS = "<eos>"

def normalize_text(text):
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = text.replace("«", '"').replace("»", '"').replace("„", '"').replace("“", '"').replace("”", '"')
    text = text.replace("—", " — ")
    text = text.replace("–", " — ")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def normalize_punct(s):
    s = s.strip()
    s = re.sub(r"\.{4,}", "...", s)
    s = re.sub(r"!{2,}", "!", s)
    s = re.sub(r"\?{2,}", "?", s)
    s = re.sub(r"(\?!){2,}", "?!", s)
    s = re.sub(r",{2,}", ",", s)
    s = re.sub(r":{2,}", ":", s)
    s = re.sub(r";{2,}", ";", s)
    s = re.sub(r"\s+([,.;:!?])", r"\1", s)
    s = re.sub(r"([,.;:!?])([^\s])", r"\1 \2", s)
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip()

def split_sentences(text):
    parts = re.split(r"(?<=[.!?])\s+", text)
    return [p for p in parts if p.strip()]

LATIN_RE = re.compile(r"[A-Za-z]")
CYR_RE = re.compile(r"[А-Яа-яЁё]")

def cyrillic_ratio(s):
    letters = re.findall(r"[A-Za-zА-Яа-яЁё]", s)
    if not letters:
        return 0.0
    cyr = sum(1 for ch in letters if CYR_RE.match(ch))
    return cyr / len(letters)

def is_good_sentence(s):
    if not s:
        return False

    if len(s) < MIN_SENT_CHARS:
        return False

    if len(s) > MAX_SENT_CHARS:
        return False

    if LATIN_RE.search(s):
        return False
    
    if cyrillic_ratio(s) < 0.70:
        return False
    
    letters_count = len(CYR_RE.findall(s))
    if letters_count < 5:
        return False
    
    if len(set(s)) <= 3:
        return False

    return True

def sha1(text):
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

def normalize_for_dedup(s):
    s = s.lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^0-9а-яё]+", "", s)
    return s.strip()

def count_tokens(text, tokenizer=None):
    if tokenizer is None:
        return len(text.split())
    return len(tokenizer.encode(text))

def chunk_sentences(sentences, max_tokens, tokenizer=None):
    chunks = []
    current = []
    current_tokens = 0

    for s in sentences:
        s_tokens = count_tokens(s, tokenizer)
        if s_tokens > max_tokens:
            words = s.split()
            buf = []
            buf_tokens = 0

            for w in words:
                w_tokens = count_tokens(w, tokenizer)
                if buf_tokens + w_tokens > max_tokens and buf:
                    chunks.append(" ".join(buf))
                    buf = [w]
                    buf_tokens = w_tokens
                else:
                    buf.append(w)
                    buf_tokens += w_tokens

            if buf:
                chunks.append(" ".join(buf))
            continue

        if current_tokens + s_tokens > max_tokens and current:
            chunks.append(" ".join(current))
            current = [s]
            current_tokens = s_tokens
        else:
            current.append(s)
            current_tokens += s_tokens
    if current:
        chunks.append(" ".join(current))

    return chunks

In [3]:
txt_files = sorted(DATA_PATH.glob("*.txt"))
print("Количество файлов:", len(txt_files))

seen_docs = set()
seen_sents = set()
all_chunks = []

stats = {
    "docs_total": 0,
    "docs_unique": 0,
    "sents_total": 0,
    "sents_good": 0,
    "sents_unique": 0,
    "chunks_total": 0
}

for fp in txt_files:
    stats["docs_total"] += 1

    raw = fp.read_text(encoding="utf-8", errors="ignore")
    raw = normalize_text(raw)

    doc_key = sha1(normalize_for_dedup(raw))
    if doc_key in seen_docs:
        continue

    seen_docs.add(doc_key)
    stats["docs_unique"] += 1

    sents = split_sentences(raw)
    stats["sents_total"] += len(sents)

    cleaned = []
    for s in sents:
        s = normalize_punct(s)
        if not is_good_sentence(s):
            continue

        stats["sents_good"] += 1

        sent_key = sha1(normalize_for_dedup(s))
        if sent_key in seen_sents:
            continue

        seen_sents.add(sent_key)
        stats["sents_unique"] += 1
        cleaned.append(s)

    chunks = chunk_sentences(cleaned, max_tokens=MAX_TOKENS_PER_CHUNK, tokenizer=None)

    for ch in chunks:
        text = f"{BOS} {ch.strip()} {EOS}"
        all_chunks.append(text)

stats["chunks_total"] = len(all_chunks)

print("=== STATS ===")
for k, v in stats.items():
    print(f"{k}: {v}")

OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
with OUT_PATH.open("w", encoding="utf-8") as f:
    for t in all_chunks:
        f.write(json.dumps({"text": t}, ensure_ascii=False) + "\n")

print("Saved:", OUT_PATH, "chunks:", len(all_chunks))

Количество файлов: 108
=== STATS ===
docs_total: 108
docs_unique: 107
sents_total: 537738
sents_good: 441048
sents_unique: 436042
chunks_total: 6386
Saved: data/pretrain_corpus.jsonl chunks: 6386


3) Токенизатор

In [4]:
import json
from pathlib import Path

JSONL_PATH = Path("./data/pretrain_corpus.jsonl")
TXT_TRAIN_PATH = Path("./data/tokenizer_train.txt")

TXT_TRAIN_PATH.parent.mkdir(parents=True, exist_ok=True)

count = 0
with open(JSONL_PATH, "r", encoding="utf-8") as f_in, open(TXT_TRAIN_PATH, "w", encoding="utf-8") as f_out:
    for line in f_in:
        obj = json.loads(line)
        text = obj["text"].strip()
        if not text:
            continue
        f_out.write(text.replace("\n", " ") + "\n")
        count += 1

print("Готово. Строк для обучения токенизатора:", count)
print("Файл:", TXT_TRAIN_PATH)

Готово. Строк для обучения токенизатора: 6386
Файл: data/tokenizer_train.txt


In [5]:
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path

VOCAB_SIZE = 3000
MIN_FREQUENCY = 2

special_tokens = ["<pad>", "<unk>", "<bos>", "<eos>"]

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(
    files=[str(TXT_TRAIN_PATH)],
    vocab_size=VOCAB_SIZE,
    min_frequency=MIN_FREQUENCY,
    special_tokens=special_tokens
)


OUT_DIR = Path("./tokenizer_bpe_3k")
OUT_DIR.mkdir(parents=True, exist_ok=True)

tokenizer.save_model(str(OUT_DIR))

tokenizer.save(str(OUT_DIR / "tokenizer.json"))
print("Saved tokenizer.json:", OUT_DIR / "tokenizer.json")




Saved tokenizer.json: tokenizer_bpe_3k/tokenizer.json


In [6]:
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset

tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="./tokenizer_bpe_3k/tokenizer.json",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>",
)

ds = load_dataset("json", data_files={"train": "./data/pretrain_corpus.jsonl"})

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        add_special_tokens=False,
        return_token_type_ids=False
    )

tokenized = ds["train"].map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

BLOCK_SIZE = 512

def group_texts(examples):
    concatenated_input_ids = []
    concatenated_attention = []

    for ids in examples["input_ids"]:
        concatenated_input_ids.extend(ids)

    for am in examples["attention_mask"]:
        concatenated_attention.extend(am)

    total_length = len(concatenated_input_ids)
    total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE

    input_ids = []
    attention_mask = []
    labels = []

    for i in range(0, total_length, BLOCK_SIZE):
        chunk_ids = concatenated_input_ids[i : i + BLOCK_SIZE]
        chunk_mask = concatenated_attention[i : i + BLOCK_SIZE]

        input_ids.append(chunk_ids)
        attention_mask.append(chunk_mask)
        labels.append(chunk_ids.copy())

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

lm_dataset = tokenized.map(
    group_texts,
    batched=True,
    remove_columns=tokenized.column_names
)

print(lm_dataset)
print("Примеров:", len(lm_dataset))
print("Длина блока:", len(lm_dataset[0]["input_ids"]))

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 6386 examples [00:00, 40259.95 examples/s]
Map: 100%|██████████| 6386/6386 [00:11<00:00, 569.19 examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26876
})
Примеров: 26876
Длина блока: 512


4. Инициализация модели

In [7]:
from transformers import PreTrainedTokenizerFast
from transformers import LlamaConfig, LlamaForCausalLM
import torch

tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="./tokenizer_bpe_3k/tokenizer.json",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>",
)

print("Vocab size:", tokenizer.vocab_size)
print("Special tokens:", tokenizer.special_tokens_map)

config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=512,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    rms_norm_eps=1e-6,
    rope_theta=10000.0,
    attention_bias=False,
)

model = LlamaForCausalLM(config)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total params:", total_params)
print("Trainable params:", trainable_params)
print("Params:", round(total_params / 1e6, 2))

model.eval()
x = torch.randint(0, tokenizer.vocab_size, (2, 32))
with torch.no_grad():
    out = model(input_ids=x)
print("Logits shape:", out.logits.shape)

Vocab size: 3000
Special tokens: {'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>'}
Total params: 132006912
Trainable params: 132006912
Params: 132.01
Logits shape: torch.Size([2, 32, 3000])


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
from datasets import Dataset
from transformers import default_data_collator

test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
]

splits = lm_dataset.train_test_split(test_size=0.02, seed=42)

train_ds = splits["train"]
val_ds = splits["test"]

print("Train size:", len(train_ds))
print("Val size:", len(val_ds))

data_collator = default_data_collator

Train size: 26338
Val size: 538


In [10]:
import torch
from transformers import TrainerCallback

class PromptGenerationCallback(TrainerCallback):
    def __init__(self, prompts, tokenizer, max_new_tokens=80):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.max_new_tokens = max_new_tokens

    def on_evaluate(self, args, state, control, model=None, **kwargs):
        if model is None:
            return

        model.eval()

        device = model.device
        print("\n" + "=" * 80)
        print(f"[Eval @ step {state.global_step}] Prompt generations")
        print("=" * 80)

        old_cache = getattr(model.config, "use_cache", False)
        model.config.use_cache = True

        with torch.no_grad():
            for i, prompt in enumerate(self.prompts):
                inputs = self.tokenizer(prompt, return_tensors="pt")
                input_ids = inputs["input_ids"].to(device)
                attention_mask = inputs["attention_mask"].to(device)

                out_ids = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=self.max_new_tokens,
                    do_sample=True,
                    temperature=0.9,
                    top_p=0.95,
                    top_k=50,
                    repetition_penalty=1.1,
                    eos_token_id=self.tokenizer.eos_token_id
                )

                text = self.tokenizer.decode(out_ids[0], skip_special_tokens=True)

                print(f"\n--- Prompt #{i+1} ---")
                print("PROMPT:", prompt)
                print("GEN:\n", text)

        model.config.use_cache = old_cache
        print("\n" + "=" * 80 + "\n")

In [11]:
from transformers import TrainingArguments, Trainer

model.config.use_cache = False
per_device_train_batch_size = 8
gradient_accumulation_steps = 8

args = TrainingArguments(
    output_dir="./checkpoints_pretrain",
    overwrite_output_dir=True,

    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,

    num_train_epochs=3,
    learning_rate=3e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",

    weight_decay=0.1,

    logging_steps=50,

    eval_strategy="steps",
    eval_steps=500,

    save_steps=500,
    save_total_limit=2,

    fp16=torch.cuda.is_available(),
    report_to="none",

    remove_unused_columns=False
)

callbacks = [PromptGenerationCallback(test_prompts, tokenizer, max_new_tokens=80)]

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=callbacks,
)

/tmp/ipykernel_19241/879651457.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
import math
trainer.train()

metrics = trainer.evaluate()
print(metrics)

if "eval_loss" in metrics:
    print("Perplexity:", math.exp(metrics["eval_loss"]))

Step,Training Loss,Validation Loss
500,3.579600,3.608132
1000,3.118000,3.316448



[Eval @ step 500] Prompt generations

--- Prompt #1 ---
PROMPT: Все мысли, которые имеют огромные последствия
GEN:
 Все мысли, которые имеют огромные последствия на лечение ее существ, - для меня все остальное, чтобы уходить директором. Их всегда, что эта история наружность и что не достигаемая фраза жизни". Он читал, но когда он остроумно обижался, она еще села в супруг, и, наконец, в сущности

--- Prompt #2 ---
PROMPT: Сила войска зависит от его духа
GEN:
 Сила войска зависит от его духа в Вене, на площадку с черным крохотным ножом. И все это были так близко: он, как всегда, спрятал у себя шпарки. Кучер, разнообразные лекции ("только не могу забывать", - со вздохами подумал Логин с подчеркнутой роскош

--- Prompt #3 ---
PROMPT: Мысль о том, что он принес страдания
GEN:
 Мысль о том, что он принес страдания в школе, что это такое же слово, -- то есть к отцу, в первых дней и в сущности соображала его; но в сущности он только хотел знать ее в жизни, как будто желая уезжать в своем деле


[Eval @ step 1236] Prompt generations

--- Prompt #1 ---
PROMPT: Все мысли, которые имеют огромные последствия
GEN:
 Все мысли, которые имеют огромные последствия, как я вижу и знал; но не в духе он испытал, когда она сказала только: "Нет, это от меня", -- и в ней никогда не приходила женщина. "А теперь я что-то говорил, -- продолжал он со вздохом, -- я видел тогда его так, как ее муж, а ты теперь". .. Он подошел к окну и спросил ее:

--- Prompt #2 ---
PROMPT: Сила войска зависит от его духа
GEN:
 Сила войска зависит от его духа, но от этого не было ни того, ни от кого. И вот, напротив, после сего я буду ходить к ней в кабинет, а то будет хорошо! Я ничего не знаю, о чем ты хочешь знать? Кого-то он убил отца, да и не должен! Ты любишь меня так, как тебе! -- Право, что я тебе

--- Prompt #3 ---
PROMPT: Мысль о том, что он принес страдания
GEN:
 Мысль о том, что он принес страдания. Попросил кучера и предложил ему пирожное место. Все эти вопросы, в которые он находился, -- он это сказал,

# Post-train SFT

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

questions_rus = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
]

def generate_answer(question):
    prompt = question
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=True,
            temperature=0.8,
            top_p=0.95,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    text = tokenizer.decode(out[0], skip_special_tokens=True)

    # уберём повтор промпта в начале (если модель его вернула)
    if text.startswith(prompt):
        text = text[len(prompt):].strip()

    return text

for i, q in enumerate(questions_rus, start=1):
    ans = generate_answer(q)
    print(f"Model Input {i}:")
    print(q)
    print(f"Model Output {i}:")
    print(ans)
    print()



`torch_dtype` is deprecated! Use `dtype` instead!


Model Input 1:
сколько планет в нашей солнечной системе?
Model Output 1:
Завод 48. Материал: Фотография и изображение, а также видео. Магазин детского питания. Детские куриные блюда для детей от 5 лет. Состав. Для детей 1 год — пакетик; для детей 2-3 года — 5 пакетиков, 6-7 месяцев — 10 пакетиков, 9 лет — 15 пакетиков. 1. Организация: Услуги, услуги.

Model Input 2:
расскажи стих
Model Output 2:
о сказке «небесная любовь» - Арифметика
Читать: 4017.382
Загадка «сказка небесная Любовь» - в этой вершине любви
Мы с дочерью открыли в гостях у Роланда, который не только любил Сашу и Хабида, но и остался на твоем берегу, и мы его пригласили к нам, чтобы посидеть с

Model Input 3:
когда собирать крыжовник?
Model Output 3:
- Страница 35
Возможно вы уже увидели подобных ряда, но я тоже не знала, как сделать крыжовник на 21 года. Недавно в интернете появился сайт, который помогает с бытовыми вопросами:
http://www.rozbrukhnik.ru/roznik/ . Купить и сделать крыжовник. В нем много советов и примеров.

## Подготовка данных

In [ ]:
from datasets import load_dataset

dataset_id = "d0rj/alpaca-cleaned-ru"
ds = load_dataset(dataset_id, split="train")

SYSTEM_PROMPT = "Ты полезный ассистент. Отвечай на русском языке кратко и по делу."

def to_messages(example):
    instruction = (example.get("instruction") or "").strip()
    inp = (example.get("input") or "").strip()
    output = (example.get("output") or "").strip()

    if inp:
        user_text = instruction + "\n\nВходные данные:\n" + inp
    else:
        user_text = instruction

    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_text},
            {"role": "assistant", "content": output},
        ]
    }

ds_chat = ds.map(to_messages, remove_columns=ds.column_names)
print(ds_chat[0]["messages"])

<|im_start|>system
Ты полезный ассистент. Отвечай на русском языке кратко и по делу.<|im_end|>
<|im_start|>user
Дайте три совета, как оставаться здоровым.<|im_end|>
<|im_start|>assistant
1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.

2. Занимайтесь регулярной физической активностью. Упражнения имеют решающее 


# Дообучение

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

def apply_template(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

ds_text = ds_chat.map(apply_template, remove_columns=["messages"])
print(ds_text[0]["text"][:400])

splits = ds_text.train_test_split(test_size=0.02, seed=42)
train_ds = splits["train"]
val_ds = splits["test"]

print("Train:", len(train_ds))
print("Val:", len(val_ds))

Map: 100%|██████████| 51760/51760 [00:06<00:00, 7699.37 examples/s]

<|im_start|>system
Ты полезный ассистент. Отвечай на русском языке кратко и по делу.<|im_end|>
<|im_start|>user
Дайте три совета, как оставаться здоровым.<|im_end|>
<|im_start|>assistant
1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм не
Train: 50724
Val: 1036


In [16]:
from trl import SFTTrainer, SFTConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

splits = ds_text.train_test_split(test_size=0.02, seed=42)
train_ds = splits["train"]
val_ds = splits["test"]

sft_args = SFTConfig(
    output_dir="./qwen2_5_0_5b_sft_ru_safe",
    overwrite_output_dir=True,

    num_train_epochs=1,
    learning_rate=5e-5,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",

    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,

    weight_decay=0.1,
    max_grad_norm=1.0,

    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,

    bf16=torch.cuda.is_available() and torch.cuda.is_bf16_supported(),
    fp16=False,

    max_length=512,
    packing=False,
    dataset_text_field="text",
    report_to="none"
)

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

trainer = SFTTrainer(
    model=model,
    args=sft_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer
)

trainer.train()


Truncating eval dataset: 100%|██████████| 1036/1036 [00:00<00:00, 167752.73 examples/s]
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss
500,1.117200,1.120233


TrainOutput(global_step=793, training_loss=1.17874399500266, metrics={'train_runtime': 1276.3682, 'train_samples_per_second': 39.741, 'train_steps_per_second': 0.621, 'total_flos': 5.384297075557171e+16, 'train_loss': 1.17874399500266})

In [17]:
import torch

SYSTEM_PROMPT = "Ты полезный ассистент. Отвечай на русском языке кратко и по делу."

def generate_answer(question, max_new_tokens=120):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(trainer.model.device)

    with torch.no_grad():
        out_ids = trainer.model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,

            do_sample=True,          # <-- включаем sampling
            temperature=0.7,
            top_p=0.9,
            top_k=50,

            repetition_penalty=1.2,  # <-- анти-повторы
            no_repeat_ngram_size=3,  # <-- запрещаем повтор 3-грамм
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    answer_ids = out_ids[0, input_ids.shape[1]:]
    return tokenizer.decode(answer_ids, skip_special_tokens=True).strip()

for i, q in enumerate(questions_rus, 1):
    ans = generate_answer(q)
    print(f"Model Input {i}:")
    print(q)
    print(f"Model Output {i}:")
    print(ans)
    print()



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_values=None`.


Model Input 1:
сколько планет в нашей солнечной системе?
Model Output 1:
СsystemsystemsystemSystemsystemsystemsapisystemsystem systemsystemsystem::systemsystem_systemsystemsystemilibriumsystemsystemscsystemsystemsystemssystemsystemserversystemsystemrogsystemsystemtestsystemsystemspacesystemsystemstationsystemsystem administrationsystemsystemstitutionssystemsystemstsystemsystem-systemsystemsystemstatsystemsystem(systemsystemsimsystemsystemstitsystemsystemEnginesystemsystemstitutesystemsystem undersystemsystemvsystemsystemverssystemsystemservicesystemsystem securitysystemsystemtrsystemsystemmainsystemsystemSystemServicesystemsystemmodelsystemsystemicularsystemsystemrecsystemsystem-systemsystemsyssystemsystemsersystemsystemstatesystemsystem-ex

Model Input 2:
расскажи стих
Model Output 2:
ВsystemsystemsystemSystemsystemsystemssystemsystem::systemsystem systemsystemsystemscsystemsystemilibriumsystemsystemstsystemsystem-exsystemsystemsystemssystemsystemservicesystemsystem_systemsystemsystem